<a href="https://colab.research.google.com/github/agiagoulas/page-stream-segmentation/blob/master/model_training/Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect to Google Drive when working in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Set working_dir

In [ ]:
working_dir = "/Tobacco800/" # TODO: Set correct working directory

Imports

In [5]:
import csv
from PIL import Image  
from os import listdir
from os.path import isfile, join
from collections import Counter
from google.colab.patches import cv2_imshow
import cv2

# Dataset Splitting

In [6]:
def splitt_tobacco800(source_dir, train_dir, test_dir):
    sorted_file_names = sorted([f for f in listdir(source_dir) if isfile(join(source_dir, f))])

    past_file_title = ''
    copy_to = "train"

    print("Copying Files to Train")
    for counter, file_name in enumerate(sorted_file_names):
        file_name_split = file_name.split('_')
        current_file_title = file_name_split[0]

        if copy_to == "train":
            if counter >= 1032 and not past_file_title == current_file_title: # rupture
                current_file_title = current_file_title.split('.')[0]
                print("Copying Files to Test")
                source = source_dir + file_name
                !cp "$source" "$test_dir"
                copy_to = "test"
            else: 
                source = source_dir + file_name
                !cp "$source" "$train_dir"
        else:
            source = source_dir + file_name
            !cp "$source" "$test_dir"
        past_file_title = current_file_title

In [7]:
tobacco800_dir = working_dir + "Tobacco800_Complete/"
train_file_dir = working_dir + "Tobacco800_Train/"
test_file_dir = working_dir + "Tobacco800_Test/"

In [ ]:
splitt_tobacco800(tobacco800_dir, train_file_dir, test_file_dir)

# Image Resizing

In [6]:
def resize(image_file):
    img = cv2.imread(image_file, 0)
    gray, img_bin = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    gray = cv2.bitwise_not(img_bin)
    dim = (255, 255)
    return cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)

In [10]:
def process_images(source_dir, target_dir):
  sorted_file_names = sorted([f for f in listdir(source_dir) if isfile(join(source_dir, f))])

  for counter, file_name in enumerate(sorted_file_names):
    source_file = source_dir + file_name
    target_file = target_dir + file_name.split(".")[0] + ".txt.png"

    image = resize(source_file)
    cv2.imwrite(target_file, image) 

    print(counter+1, "of", len(sorted_file_names))
    print("File name:", file_name)
    print("Target:", target_file)

In [8]:
source_train_files = working_dir + "Tobacco800_Train/"
source_test_files = working_dir + "Tobacco800_Test/"
small_file_dir = working_dir + "Tobacco800_Small/"

In [ ]:
process_images(source_test_files, small_file_dir)

In [ ]:
process_images(source_train_files, small_file_dir)